In [2]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/democompute/code/Users/callkabhishek/diabetes'

In [1]:
from azureml.core import Workspace
import azureml.core
import os,json,sys
from azureml.core.authentication import InteractiveLoginAuthentication

print('SDK Version:', azureml.core.VERSION)

with open("./aml_config/config.json") as f:
    config=json.load(f)

workspace_name=config["workspace_name"]
print(workspace_name)

resource_group=config["resource_group"]
subscription_id=config["subscription_id"]

inter_auth=InteractiveLoginAuthentication()
#ws = Workspace.from_config()

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id,resource_group=resource_group,auth=inter_auth) 
print(ws.name, ws.resource_group,ws.subscription_id)



SDK Version: 1.42.0
mlops-demo
mlops-demo rg-mlops-demo 2279dc50-0d1e-45f3-8ba1-6832bc4b5b02


In [4]:
from azureml.core import Experiment
experiment = Experiment(workspace=ws, name="diabetes-experiment")

In [5]:
from azureml.opendatasets import Diabetes
from sklearn.model_selection import train_test_split

x_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna()
y_df = x_df.pop("Y")

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=66)


In [6]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import math

alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for alpha in alphas:
    run = experiment.start_logging()
    run.log("alpha_value", alpha)
    
    model = Ridge(alpha=alpha)
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X=X_test)
    rmse = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))
    run.log("rmse", rmse)
    
    model_name = "model_alpha_" + str(alpha) + ".pkl"
    filename = "outputs/" + model_name
    
    joblib.dump(value=model, filename=filename)
    run.upload_file(name=model_name, path_or_stream=filename)
    run.complete()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [7]:
experiment

Name,Workspace,Report Page,Docs Page
diabetes-experiment,mlops-demo,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
minimum_rmse_runid = None
minimum_rmse = None

for run in experiment.get_runs():
    run_metrics = run.get_metrics()
    run_details = run.get_details()
    # each logged metric becomes a key in this returned dict
    run_rmse = run_metrics["rmse"]
    run_id = run_details["runId"]
    
    if minimum_rmse is None:
        minimum_rmse = run_rmse
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse:
            minimum_rmse = run_rmse
            minimum_rmse_runid = run_id

print("Best run_id: " + minimum_rmse_runid)
print("Best run_id rmse: " + str(minimum_rmse))    


Best run_id: de0c3bf9-5adb-42f3-aa06-f7a9d4ce8481
Best run_id rmse: 56.605203313391435


In [10]:
from azureml.core import Run
best_run = Run(experiment=experiment, run_id=minimum_rmse_runid)
print(best_run.get_file_names())


# Call `download()` on the run object, specifying the model file name to download. By default this function downloads to the current directory.

# In[9]:

best_run.download_file(name="model_alpha_0.1.pkl")
print("Best performing experiment run identified")

print("Registering best run model with the Azureml workspace")
model = best_run.register_model(model_name='diabetes-model', 
                                model_path='model_alpha_0.1.pkl')

['model_alpha_0.1.pkl', 'outputs/.amlignore', 'outputs/.amlignore.amltmp', 'outputs/model_alpha_0.1.pkl', 'outputs/model_alpha_0.2.pkl', 'outputs/model_alpha_0.3.pkl', 'outputs/model_alpha_0.4.pkl', 'outputs/model_alpha_0.5.pkl', 'outputs/model_alpha_0.6.pkl', 'outputs/model_alpha_0.7.pkl', 'outputs/model_alpha_0.8.pkl', 'outputs/model_alpha_0.9.pkl', 'outputs/model_alpha_1.0.pkl']
Best performing experiment run identified
Registering best run model with the Azureml workspace


In [11]:
model.get_model_path
#model.name

<function azureml.core.model.Model.get_model_path(model_name, version=None, _workspace=None)>

In [12]:
print ("Creating deploy environment yaml file")
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.image import ContainerImage

# Create an empty conda environment and add the scikit-learn package
env = CondaDependencies()
env.add_conda_package("scikit-learn")

# Display the environment
print(env.serialize_to_string())

# Write the environment to disk
with open("myenv.yml","w") as f:
    f.write(env.serialize_to_string())



Creating deploy environment yaml file
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.8.12

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [29]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
#prov_config = AksCompute.provisioning_configuration(location="westeurope",vm_size="Standard_DS3_V2")
prov_config = AksCompute.provisioning_configuration(vm_size="Standard_DS2_V2")

aks_cluster_name = "mlopscluster"
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)


InProgress..................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None


In [13]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.exceptions import ComputeTargetException

# Choose a name for your cluster
aks_cluster_name = "mlopscluster"

# Check to see if the cluster already exists
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')

    aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)


    aks_target.wait_for_completion(show_output=True)
    print(aks_target.provisioning_state)
    print(aks_target.provisioning_errors)



Found existing compute target


In [14]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python",
                                              #source_directory="script_folder",
                                              entry_script="score.py",
                                              conda_file="myenv.yml")


In [15]:
aks_target = AksCompute(ws,"mlopscluster")
service_name="diabetes-svc-v1"


In [85]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute
from azureml.core.model import Model


deployment_config = AksWebservice.deploy_configuration(cpu_cores = 0.5, memory_gb = 2)
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)

service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry acrmlopsdemo1.azurecr.io
Logging into Docker registry acrmlopsdemo1.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM acrmlopsdemo1.azurecr.io/azureml/azureml_a4ef8c288de4e4de116146287923748e
 ---> cee3f1012804
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> a8d6cf37b20e
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjIyNzlkYzUwLTBkMWUtNDVmMy04YmExLTY4MzJiYzRiNWIwMiIsInJlc291cmNlR3JvdXBOYW1lIjoicmctbWxvcHMtZGVtbyIsImFjY291bnROYW1lIjoibWxvcHMtZGVtbyIsIndvcmtzcGFjZUlkIjoiNTY0NWMyZGYtOTBhOS00MDQ5LWI2MGItOTFmYTUzNWJhNzhhIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 5fb3ecfe4a63
 ---> 6e343e9a1da6
Step 4/5 : RUN mv '/var/azureml-app/tmp3gv2c2r7.py' /var/azureml-app/main.py
 ---> Running in 03a9f8da8e41
 ---> 9d29a76f8079
Step 5/5 : CMD ["runsvdir","/var/run

In [40]:
local_service.get_logs()

'/bin/bash: /azureml-envs/azureml_cdac53350582a78a48f29cd457ee19c9/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n2022-08-11T03:27:27,221107835+00:00 - gunicorn/run \n2022-08-11T03:27:27,222992266+00:00 | gunicorn/run | \n2022-08-11T03:27:27,227171457+00:00 - iot-server/run \n/bin/bash: /azureml-envs/azureml_cdac53350582a78a48f29cd457ee19c9/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_cdac53350582a78a48f29cd457ee19c9/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_cdac53350582a78a48f29cd457ee19c9/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n2022-08-11T03:27:27,231338247+00:00 - rsyslog/run \nbash: /azureml-envs/azureml_cdac53350582a78a48f29cd457ee19c9/lib/libtinfo.so.6: no version information available (required by bash)\n2022-08-11T03:27:27,234480465+00:00 | gunicorn/run | ##########################

In [16]:
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()
print(local_service.state)


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry acrmlopsdemo1.azurecr.io
Logging into Docker registry acrmlopsdemo1.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM acrmlopsdemo1.azurecr.io/azureml/azureml_a4ef8c288de4e4de116146287923748e
 ---> cee3f1012804
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> b4f30deabf06
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjIyNzlkYzUwLTBkMWUtNDVmMy04YmExLTY4MzJiYzRiNWIwMiIsInJlc291cmNlR3JvdXBOYW1lIjoicmctbWxvcHMtZGVtbyIsImFjY291bnROYW1lIjoibWxvcHMtZGVtbyIsIndvcmtzcGFjZUlkIjoiNTY0NWMyZGYtOTBhOS00MDQ5LWI2MGItOTFmYTUzNWJhNzhhIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in f47aa7d616f7
 ---> f69e6218ab7b
Step 4/5 : RUN mv '/var/azureml-app/tmpjomp39pr.py' /var/azureml-app/main.py
 ---> Running in 241492b7f343
 ---> 525e1af43d5c
Step 5/5 : CMD ["runsvdir","/var/run

Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}